In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm # Anzeigen des aktuellen Ladestandes
import PyPDF2
import re
from os import listdir
from os.path import isfile, join
import locale #für das deutsche Zeitformat

# 5. Alle Dataframes zusammenfügen 
- Alle einlesen und auf die Unterseiten verweisen
- eines nach dem andern anhängen
- noch schön machen und kontrollieren

In [2]:
# Einlesen:
df_v=pd.read_csv("KRFrames/df_v_Hauptseiten") #Alle Vorstösse mit Infos mit Titel und Nummer der Page mit allen Geschäften
df_dat_all=pd.read_csv("KRFrames/df_datum_ALLE")# Frame mit allen Zeitangaben(Datum)(oben)
df_np=pd.read_csv("KRFrames/Frame_Namen_Partei_endversion")# Aus File "02 Namen aus Geschäftsnamen..." Alle Namen und Parteien



# nun mal alle gleich formatieren 

In [3]:
df_v.dtypes #würde sagen das ist ok und unnamed wegnehmen

Unnamed: 0            int64
level_0               int64
index                 int64
VorlageNR             int64
url                  object
Geschäft             object
Stand                object
Zuständigkeit        object
Art des Geschäfts    object
Status               object
dtype: object

In [4]:
df_v1=df_v[["VorlageNR", "Geschäft", "Art des Geschäfts", "Zuständigkeit", "Status", "Stand", "url"]]

In [5]:
df_v1[df_v1["Geschäft"].str.contains("Kommission")] # Ok, offenbar hat es im df_v die Kommissionsgeschäfte noch
# Die hatte ich beim anderen Dataframe rausgeputzt.

,VorlageNR,Geschäft,Art des Geschäfts,Zuständigkeit,Status,Stand,url
416,2412,Motion der Kommission zur Totalrevision der Ge...,Motion,Staatskanzlei,abgeschlossen,Erledigterklärung,/gast/geschaefte/1399
419,2408,Interpellation der ad-hoc Kommission zur Unter...,Interpellation,Finanzdirektion,abgeschlossen,Kenntnisnahme,/gast/geschaefte/1389
420,2407,Motion der ad-hoc Kommission zur Untersuchung ...,Motion,Finanzdirektion,abgeschlossen,Erledigterklärung,/gast/geschaefte/1388
530,2243,Motion der vorberatenden Kommission zum Pensio...,Motion,Finanzdirektion,abgeschlossen,Erledigterklärung,/gast/geschaefte/494
534,2238,Motion der vorberatenden Kommission zur Änderu...,Motion,Finanzdirektion,abgeschlossen,Erledigterklärung,/gast/geschaefte/489
554,2202,Motion der vorberatenden Kommission zum Integr...,Motion,Direktion für Bildung und Kultur,abgeschlossen,Erledigterklärung,/gast/geschaefte/454
599,2124,Motion der Kommission Polycom zum Kantonsratsb...,Motion,Sicherheitsdirektion,abgeschlossen,Erledigterklärung,/gast/geschaefte/376
604,2114,Motion von Thomas Aeschi betreffend Möglichkei...,Motion,Sicherheitsdirektion,abgeschlossen,Nicht-Erheblicherklärung,/gast/geschaefte/366
707,1910,Motion der erweiterten Justizprüfungskommissio...,Motion,Staatskanzlei,abgeschlossen,Erledigterklärung,/gast/geschaefte/171
984,1447,Motion der Kommission für Wasserbau und Gewäss...,Motion,NaN,abgeschlossen,Datenmigration,/gast/geschaefte/1032


In [6]:
df_dat_all.dtypes # Da sollte ich wohl datetime einstellen und die unamed wegnehmen

Unnamed: 0            int64
Unnamed: 0_x          int64
VorlageNR_x           int64
Einreichedatum       object
Link Vorstoss-PDF    object
dtype: object

In [7]:
df_dat_all["Einreichedatum"]=pd.to_datetime(df_dat_all["Einreichedatum"])

In [8]:
df_dat_all.dtypes 

Unnamed: 0                    int64
Unnamed: 0_x                  int64
VorlageNR_x                   int64
Einreichedatum       datetime64[ns]
Link Vorstoss-PDF            object
dtype: object

In [9]:
df_dat_all1=df_dat_all[["VorlageNR_x","Einreichedatum"]]
df_dat_all2=df_dat_all1.rename(columns={"VorlageNR_x": "VorlageNR"})

In [10]:
# ich mache das mit den Vorstössen mal einzeln
df_pdf_links=df_dat_all[["VorlageNR_x","Link Vorstoss-PDF"]]
df_pdf_links2=df_pdf_links.rename(columns={"VorlageNR_x": "VorlageNR"})

In [11]:
df_np.dtypes 
# Alles wegnehmen, ausser Namenganz, Partei und VorlageNR

Unnamed: 0          int64
Namenganz          object
P                  object
Unnamed: 0.1      float64
Unnamed: 0.1.1    float64
VorlageNR           int64
dtype: object

In [12]:
df_np1=df_np[["VorlageNR", "Namenganz", "P"]]

In [13]:
# Nun zusammenfügen
df_hau_dat=pd.merge(df_v1, df_dat_all2, how="inner",on= "VorlageNR")

In [14]:
df_fehlende_daten=df_hau_dat.sort_values("Einreichedatum").tail(24)

In [15]:
df_fe_da_mi_pdf=pd.merge(df_fehlende_daten, df_pdf_links2, how="inner",on= "VorlageNR")

In [16]:
df_fehlende=df_fe_da_mi_pdf.tail(21)

In [17]:
# Hier putze ich noch die letzte Zahl raus. die Kein PDF hat.
df_fehlende_s=df_fehlende[df_fehlende["Link Vorstoss-PDF"].str.contains("183")==False]

In [18]:
# Nun einen Loop bauen, um auch in diesen PDFs die Daten rauszukriegen.Zuerst an einem Testen

# Einzelne PDF durchschauen

In [31]:
dat_pdf=[]
mypdf1 = open("KRGeschPDF/"+"13075_1817_1_Strassenbau.pdf", mode='rb')
pdf_document1 = PyPDF2.PdfFileReader(mypdf1)
first_page1 = pdf_document1.getPage(0)
front1=first_page1.extractText()
print(front1)
dat_p_ein1= re.findall(regex_datum, front1, re.IGNORECASE)[0]
dat_p_zwe1= re.findall(regex_datum1, front1, re.IGNORECASE)[0]
dat_p_dre1= re.findall(regex_datum2, front1, re.IGNORECASE)[0]
    #Ich greiffe auf das Datum zu
        # um zu verhindern, dass ich mehrer Daten und Vorlagen habe, sage ich mit [0] er soll das erste nehmen
vorl_p_nr1= re.findall(regex_vorlage2, front1, re.IGNORECASE)[0] # Ich greiffe auf die Vorlagennummer zu (zum Zusammenfügen der Dataframes)
    
    
minidict_pdf={"Einreichedatum": dat_p_ein1, "Einreichedatum2": dat_p_zwe1, "Einreichedatum3": dat_p_dre1, "VorlageNR":vorl_p_nr1, }
    
dat_pdf.append(minidict_pdf)
dat_pdf

[{'Einreichedatum': '',
  'Einreichedatum2': '',
  'Einreichedatum3': '',
  'VorlageNR': ''}]

# Hier die Lösung

In [25]:
regex_vorlage= r"Vorlage.Nr\..(\d*)\.1|$" #hier ziehe ich die Vorlagennummer raus. 
regex_vorlage1= r"Vorlage.Nr\..(\d*)\|$"
regex_vorlage2= r"(\d*)\.1|$" 
regex_datum = r"vom.(\d+\..\w*.\d{4})|$" # hier gehe ich an das DAtum
regex_datum2 =r"(\d*\..\w*.\d{4})|$"
regex_datum1 = r"VOM.(\d+\..\w*.\d{4})|$" # hier gehe ich an das DAtum
regex_name=r"\bVON(.*UND.*)" #Hier muss ich noch überlegen, wie anders. Problem: zum Teil sind Namen einzeln zum teil mehr
# Und irgendwie erkennt er die Zeilenende nicht.
regex_name1= r"\bVON.(\w*.\w*\b)"
regex_partei= r"DER.(\w*)"

In [35]:
dat_pdf_list=[]
for dok in df_fehlende_s["Link Vorstoss-PDF"]:
    mypdf = open("KRGeschPDF/"+dok, mode='rb')
    pdf_name = dok
    pdf_document = PyPDF2.PdfFileReader(mypdf)
    first_page = pdf_document.getPage(0)
    front=first_page.extractText()
    dat_p_dre= re.findall(regex_datum2, front, re.IGNORECASE)[0]
    #Ich greiffe auf das Datum zu
        # um zu verhindern, dass ich mehrer Daten und Vorlagen habe, sage ich mit [0] er soll das erste nehmen
    vorl_p_nr= re.findall(regex_vorlage2, front, re.IGNORECASE)[0] # Ich greiffe auf die Vorlagennummer zu (zum Zusammenfügen der Dataframes)
    
    
    minidict_pdf_dat={"Einreichedatum3": dat_p_dre, "VorlageNR":vorl_p_nr, "Link Vorstoss-PDF":pdf_name}
    
    dat_pdf_list.append(minidict_pdf_dat)

In [36]:
df_pdf2=pd.DataFrame(dat_pdf_list)

In [38]:
df_pdf2.to_csv("KRFrames/fehlende_Daten") #ok, geht irgendwie nicht. Mal mit einem Einzelnen pdf probieren

In [ ]:
df_all=pd.merge(df_v1, df_dat_all, how="outer",on= "VorlageNR")

In [ ]:
# Offenbar hat es einzelne Vorlagenr doppelt mit mehreren Daten. ich kick die mal weg. 

df_allgut=df_all.drop_duplicates(subset='VorlageNR', keep=False) 

In [ ]:
df_allgut.sort_values("Einreichedatum").tail(100)

In [ ]:
df_alles.shape

In [ ]:
df_alles.to_csv("KRFrames/df_alles_ohne_Namen")

In [ ]:
df_np

In [ ]:
df_alles_ganz=pd.merge(df_alles, df_np, how="left", on="VorlageNR")
df_alles_ganz.sort_values("Einreichedatum")

In [ ]:
df_tuti=df_alles_ganz[["VorlageNR","Geschäft", "Namenganz", "P", "Einreichedatum", "Link Vorstoss-PDF"]]

In [ ]:
df_tuti #Wieso wird es nicht ergänzt. Der Fehler ist im File 02 Namen passiert. dort nachschauen.

In [ ]:

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 100)
df_tuti1=df_tuti.drop_duplicates(subset=['VorlageNR', 'Namenganz'], keep=False)
# Obwohl mir nur von 30 Geschäften das Datum fehlt, explodiert dies, wenn es auf die Namen-Tabelle kommt.
df_tuti1.sort_values("Einreichedatum").tail(188)

In [ ]:
df_tuti.to_csv("KRFrames/df_tuti_Ganzes_Frame")

# Daten aus B&A mit falschen Zahlen
Eine Lösung wäre, nochmals die Links zu den PDF auslesen und diese in zwei verschiedene Listen verpacken. 
Da die Datenmigration im Jahr 2000 war, schaue ich einfach mal die Zahlen ab 2005 an. Vo dort an, werden die meisten Geschäfte tatsächlich neu sein. 
## #Andere Variante: ich lasse die B&A einfach weg. Da gibt es eh keine verlässliche Daten. Mengenmässig.
# Nun plotten

In [ ]:
df_tuti = df_tuti.set_index('Einreichedatum')

In [ ]:
df_tuti

In [ ]:
df_tuti.dtypes